# Basic setup


In [1]:
pip install -U plotly


     |████████████████████████████████| 13.2MB 329kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [2]:
import pandas as pd 
import numpy as np 
import statsmodels.api as sm
from datetime import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import seaborn as sns
from urllib.request import urlopen
import json
import plotly
from google.colab import files
print(plotly.__version__)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


4.14.1


In [3]:
# FIPS CODES: https://github.com/moon1ock/AQI_vs_Corona/blob/main/data/fips_codes.csv

# State names: https://github.com/moon1ock/AQI_vs_Corona/blob/main/data/state_names.csv
"""
Aggregating FIPS and State names on State abbr

df_fips = pd.read_csv('https://raw.githubusercontent.com/jaisal1024/coronavirus-us-testing/master/data/fips_codes.csv')
df = pd.merge(df, df_fips, on ='state')


df_names = pd.read_csv('https://raw.githubusercontent.com/moon1ock/AQI_vs_Corona/main/data/state_names.csv')
df = pd.merge(df, df_names, left_on = 'state', right_on = 'State', how = 'left')

"""

"\nAggregating FIPS and State names on State abbr\n\ndf_fips = pd.read_csv('https://raw.githubusercontent.com/jaisal1024/coronavirus-us-testing/master/data/fips_codes.csv')\ndf = pd.merge(df, df_fips, on ='state')\n\n\ndf_names = pd.read_csv('https://raw.githubusercontent.com/moon1ock/AQI_vs_Corona/main/data/state_names.csv')\ndf = pd.merge(df, df_names, left_on = 'state', right_on = 'State', how = 'left')\n\n"

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/moon1ock/AQI_vs_Corona/main/data/clean_data.csv')

In [5]:
df.head()

,Unnamed: 0,date,Name,state,FIPS,positive,AQI,date_str
0,785,2020-01-20,Massachusetts,MA,25,0,34.333333,01/20/2020
1,1950,2020-01-20,Washington,WA,53,0,35.000000,01/20/2020
2,1951,2020-01-27,Washington,WA,53,0,38.000000,01/27/2020
3,786,2020-01-27,Massachusetts,MA,25,0,31.714286,01/27/2020
4,369,2020-01-27,Florida,FL,12,0,44.666667,01/27/2020


In [6]:
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d').dt.strftime('%m/%d/%Y')


In [7]:
df.head()

,Unnamed: 0,date,Name,state,FIPS,positive,AQI,date_str
0,785,01/20/2020,Massachusetts,MA,25,0,34.333333,01/20/2020
1,1950,01/20/2020,Washington,WA,53,0,35.000000,01/20/2020
2,1951,01/27/2020,Washington,WA,53,0,38.000000,01/27/2020
3,786,01/27/2020,Massachusetts,MA,25,0,31.714286,01/27/2020
4,369,01/27/2020,Florida,FL,12,0,44.666667,01/27/2020


In [8]:
df_clean = df[['date', 'Name', 'state', 'FIPS', 'positive', 'AQI']]

In [9]:
df_clean.sort_values(by=["date", "state"])

,date,Name,state,FIPS,positive,AQI
0,01/20/2020,Massachusetts,MA,25,0,34.333333
1,01/20/2020,Washington,WA,53,0,35.000000
4,01/27/2020,Florida,FL,12,0,44.666667
3,01/27/2020,Massachusetts,MA,25,0,31.714286
2,01/27/2020,Washington,WA,53,0,38.000000
...,...,...,...,...,...,...
2091,12/07/2020,Vermont,VT,50,5180,36.142857
2088,12/07/2020,Washington,WA,53,184404,36.285714
2094,12/07/2020,Wisconsin,WI,55,448009,35.428571
2107,12/07/2020,West Virginia,WV,54,57060,35.142857


In [10]:
sns.set()

In [11]:
# fig = px.choropleth(df_clean.loc[df_clean["date"] == dt.strptime('2020-10-01', '%Y-%m-%d')][["state", "positive", "FIPS"]], 
#                            color='positive',
#                            locationmode = 'USA-states',
#                            locations='state',
#                            color_continuous_scale="thermal",
#                            hover_name = 'state',
#                            scope = 'usa',
#                            labels={'tests_per_1000':'Tests Per 1000 People'}, 
#                            title = "US State-By-State Tests per 1000 People on {}".format('2020-05-02')
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

# Let's make a real time animation

In [12]:
# sort on datetime 
# we'll need to sort and convert to str for regressions later
df_clean.sort_values(by=["date", "state"])

,date,Name,state,FIPS,positive,AQI
0,01/20/2020,Massachusetts,MA,25,0,34.333333
1,01/20/2020,Washington,WA,53,0,35.000000
4,01/27/2020,Florida,FL,12,0,44.666667
3,01/27/2020,Massachusetts,MA,25,0,31.714286
2,01/27/2020,Washington,WA,53,0,38.000000
...,...,...,...,...,...,...
2091,12/07/2020,Vermont,VT,50,5180,36.142857
2088,12/07/2020,Washington,WA,53,184404,36.285714
2094,12/07/2020,Wisconsin,WI,55,448009,35.428571
2107,12/07/2020,West Virginia,WV,54,57060,35.142857


In [13]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121 entries, 0 to 2120
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      2121 non-null   object 
 1   Name      2121 non-null   object 
 2   state     2121 non-null   object 
 3   FIPS      2121 non-null   int64  
 4   positive  2121 non-null   int64  
 5   AQI       2111 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 99.5+ KB


In [14]:
df_clean['date'] = pd.to_datetime(df_clean['date'], format='%m/%d/%Y')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
df_clean['date_str'] = df_clean['date'].apply(lambda x: dt.strftime(x, "%m/%d/%Y"))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [16]:
df_clean

,date,Name,state,FIPS,positive,AQI,date_str
0,2020-01-20,Massachusetts,MA,25,0,34.333333,01/20/2020
1,2020-01-20,Washington,WA,53,0,35.000000,01/20/2020
2,2020-01-27,Washington,WA,53,0,38.000000,01/27/2020
3,2020-01-27,Massachusetts,MA,25,0,31.714286,01/27/2020
4,2020-01-27,Florida,FL,12,0,44.666667,01/27/2020
...,...,...,...,...,...,...,...
2116,2020-12-07,Iowa,IA,19,214897,39.571429,12/07/2020
2117,2020-12-07,South Carolina,SC,45,236954,37.142857,12/07/2020
2118,2020-12-07,Hawaii,HI,15,18976,22.428571,12/07/2020
2119,2020-12-07,Illinois,IL,17,804174,49.285714,12/07/2020


In [17]:
fig = px.choropleth(df_clean, 
                           color='positive',
                           locationmode = 'USA-states',
                           locations='state',
                           color_continuous_scale="thermal_r",
                           hover_name = 'Name',
                           hover_data = ['AQI'],
                           scope = 'usa',
                           animation_frame = 'date_str',
                           animation_group = 'date_str', 
                           labels={'AQI':'AQI'}, 
                           title = "US State-By-State Positive Cases"  )
fig.update_layout(height=500,margin={"r":0,"t":45,"l":0,"b":0})
fig1 = px.choropleth(df_clean, 
                           color='AQI',
                           locationmode = 'USA-states',
                           locations='state',
                           color_continuous_scale="thermal_r",
                           hover_name = 'Name',
                           hover_data = ['positive'],
                           scope = 'usa',
                           animation_frame = 'date_str',
                           animation_group = 'date_str', 
                           title = "US State-By-State AQI indices"  )
fig1.update_layout(height=500, margin={"r":0,"t":45,"l":0,"b":0})
fig.show()
fig1.show()
## FLIP Color code 

In [18]:
# df_clean.to_csv("tograph.csv")
# files.download("tograph.csv")

# Adjusted Spearman's rank correlation


In [19]:
df_clean.describe()

,FIPS,positive,AQI
count,2121.000000,2.121000e+03,2111.000000
mean,29.016973,6.098757e+05,42.657225
std,15.670839,1.072515e+06,14.601894
min,1.000000,0.000000e+00,2.000000
25%,16.000000,3.299800e+04,35.000000
50%,29.000000,1.995200e+05,41.142857
75%,42.000000,7.347880e+05,48.142857
max,56.000000,9.041993e+06,153.285714


In [20]:
df_clean["rank"] =  df_clean[['positive', 'AQI']].apply(lambda x: x[0]/(9.041993e+06) + x[1]/(153.285714) , axis=1)

In [21]:
fig = px.choropleth(df_clean, 
                           color='rank',
                           locationmode = 'USA-states',
                           locations='state',
                           color_continuous_scale="thermal_r",
                           hover_name = 'Name',
                           hover_data = ['AQI', 'positive'],
                           scope = 'usa',
                           animation_frame = 'date_str',
                           animation_group = 'date_str', 
                           title = "State by Rank"  )
fig.update_layout(height=500,margin={"r":0,"t":45,"l":0,"b":0})
fig.show()

In [22]:
df_clean.loc[df_clean['state'] == "MO"]

,date,Name,state,FIPS,positive,AQI,date_str,rank
53,2020-03-02,Missouri,MO,29,2,34.666667,03/02/2020,0.226157
89,2020-03-09,Missouri,MO,29,20,28.285714,03/09/2020,0.184532
176,2020-03-16,Missouri,MO,29,438,36.142857,03/16/2020,0.235836
200,2020-03-23,Missouri,MO,29,4417,52.428571,03/23/2020,0.342520
240,2020-03-30,Missouri,MO,29,14235,44.000000,03/30/2020,0.288620
327,2020-04-06,Missouri,MO,29,26274,33.142857,04/06/2020,0.219122
368,2020-04-13,Missouri,MO,29,36966,30.000000,04/13/2020,0.199801
420,2020-04-20,Missouri,MO,29,46018,28.142857,04/20/2020,0.188687
450,2020-04-27,Missouri,MO,29,55419,24.714286,04/27/2020,0.167359
509,2020-05-04,Missouri,MO,29,66276,40.714286,05/04/2020,0.272940


# Causal inferencing and correlations


## Plot the top 10 covid states against all States

In [23]:
top_10_states = df_clean.loc[df_clean["date"] == dt.strptime('2020-09-21', '%Y-%m-%d')].sort_values(by = 'positive').tail(10)['state']

In [24]:
top_10_states

1532    TN
1549    NJ
1557    NC
1525    AZ
1521    IL
1520    GA
1539    NY
1534    FL
1509    TX
1554    CA
Name: state, dtype: object

In [25]:
df_top10_select = df_clean.loc[(df_clean["state"].isin(top_10_states)) & (df_clean["date"] >= dt.strptime('2020-03-15', '%Y-%m-%d')),].copy()

In [26]:
df_top10_select.head()

,date,Name,state,FIPS,positive,AQI,date_str,rank
134,2020-03-16,New York,NY,36,61410,42.142857,03/16/2020,0.281722
137,2020-03-16,California,CA,6,7629,75.571429,03/16/2020,0.493854
139,2020-03-16,Tennessee,TN,47,2044,39.714286,03/16/2020,0.259313
149,2020-03-16,Florida,FL,12,3344,36.142857,03/16/2020,0.236157
152,2020-03-16,Illinois,IL,17,4529,52.714286,03/16/2020,0.344397


In [27]:
fig = px.scatter(df_clean, x="date_str", y="AQI",
	               size="positive", color="state",
                 hover_name="Name", size_max=60,
                 title = "Correlation of AQI vs Positives in Bubbles")
fig.show()

In [28]:
fig = px.scatter(df_top10_select, x="date_str", y="AQI",
	               size="positive", color="state",
                 hover_name="Name", size_max=60,
                 title = "Correlation of AQI vs Positives in Bubbles")
fig.show()

## Looking into the timeless numbers

In [29]:
fig = px.scatter(df_top10_select, x="positive", y="AQI",
               color="state",
                 hover_name="Name", size_max=60,
                 title = "Correlation of AQI vs Positives with no 3d dimension")
fig.show()

## Regressions

In [30]:
df_top10_select

,date,Name,state,FIPS,positive,AQI,date_str,rank
134,2020-03-16,New York,NY,36,61410,42.142857,03/16/2020,0.281722
137,2020-03-16,California,CA,6,7629,75.571429,03/16/2020,0.493854
139,2020-03-16,Tennessee,TN,47,2044,39.714286,03/16/2020,0.259313
149,2020-03-16,Florida,FL,12,3344,36.142857,03/16/2020,0.236157
152,2020-03-16,Illinois,IL,17,4529,52.714286,03/16/2020,0.344397
...,...,...,...,...,...,...,...,...
2101,2020-12-07,Florida,FL,12,1056065,54.428571,12/07/2020,0.471875
2102,2020-12-07,Tennessee,TN,47,414749,37.000000,12/07/2020,0.287249
2103,2020-12-07,Georgia,GA,13,452369,29.142857,12/07/2020,0.240151
2114,2020-12-07,North Carolina,NC,37,404032,30.285714,12/07/2020,0.242261


In [31]:
fig = px.scatter(df_top10_select, x="positive", y="AQI", color = "state", trendline = "ols", hover_name = "Name", title = "OLS timeless positive vs AQI")
fig.show()

In [32]:
fig = px.scatter(df_top10_select, 
                 x="rank", y="AQI", 
                 color = "state", trendline = "ols",
                 hover_name = "Name", title = "OLS timeless RANK vs AQI")
fig.show()

Note that CA r^2 is 46%, while FL has 96% of variability explained

### Since it's so fun, let's look at the statistics of these

In [33]:
results = px.get_trendline_results(fig)
print(results)

  state                                     px_fit_results
0    NY  <statsmodels.regression.linear_model.Regressio...
1    CA  <statsmodels.regression.linear_model.Regressio...
2    TN  <statsmodels.regression.linear_model.Regressio...
3    FL  <statsmodels.regression.linear_model.Regressio...
4    IL  <statsmodels.regression.linear_model.Regressio...
5    GA  <statsmodels.regression.linear_model.Regressio...
6    TX  <statsmodels.regression.linear_model.Regressio...
7    NJ  <statsmodels.regression.linear_model.Regressio...
8    NC  <statsmodels.regression.linear_model.Regressio...
9    AZ  <statsmodels.regression.linear_model.Regressio...


In [34]:
results.query("state == 'NY'").px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.015
Method:                 Least Squares   F-statistic:                    0.4492
Date:                Thu, 10 Dec 2020   Prob (F-statistic):              0.507
Time:                        17:44:40   Log-Likelihood:                -142.72
No. Observations:                  39   AIC:                             289.4
Df Residuals:                      37   BIC:                             292.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.8210      6.898      2.438      0.020       2.844      30.798
x1            10.2804     15.339      0.670      0.507     -20.800      41.361
==============================================================================
Omnibus:                        3.318   Durbin-Watson:                   1.542
Prob(Omnibus):                  0.190   Jarque-Bera (JB):                2.921
Skew:                           0.583   Prob(JB):                        0.232
Kurtosis:                       2.337   Cond. No.                         11.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
results.query("state == 'CA'").px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.461
Model:                            OLS   Adj. R-squared:                  0.446
Method:                 Least Squares   F-statistic:                     31.63
Date:                Thu, 10 Dec 2020   Prob (F-statistic):           2.03e-06
Time:                        17:44:40   Log-Likelihood:                -172.06
No. Observations:                  39   AIC:                             348.1
Df Residuals:                      37   BIC:                             351.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         36.3574      8.043      4.520      0.000      20.060      52.655
x1            47.2822      8.408      5.624      0.000      30.247      64.318
==============================================================================
Omnibus:                        5.444   Durbin-Watson:                   1.863
Prob(Omnibus):                  0.066   Jarque-Bera (JB):                5.791
Skew:                           0.295   Prob(JB):                       0.0553
Kurtosis:                       4.793   Cond. No.                         4.70
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
results.query("state == 'TX'").px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.8387
Date:                Thu, 10 Dec 2020   Prob (F-statistic):              0.366
Time:                        17:44:40   Log-Likelihood:                -146.38
No. Observations:                  39   AIC:                             296.8
Df Residuals:                      37   BIC:                             300.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         48.3982      4.194     11.541      0.000      39.901      56.896
x1             5.2380      5.720      0.916      0.366      -6.351      16.827
==============================================================================
Omnibus:                        4.067   Durbin-Watson:                   1.301
Prob(Omnibus):                  0.131   Jarque-Bera (JB):                2.863
Skew:                           0.502   Prob(JB):                        0.239
Kurtosis:                       2.131   Cond. No.                         4.98
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [37]:
results.query("state == 'FL'").px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                    0.1313
Date:                Thu, 10 Dec 2020   Prob (F-statistic):              0.719
Time:                        17:44:40   Log-Likelihood:                -135.73
No. Observations:                  39   AIC:                             275.5
Df Residuals:                      37   BIC:                             278.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         43.9532      3.280     13.400      0.000      37.307      50.599
x1            -1.8561      5.123     -0.362      0.719     -12.237       8.525
==============================================================================
Omnibus:                        6.457   Durbin-Watson:                   2.093
Prob(Omnibus):                  0.040   Jarque-Bera (JB):                2.419
Skew:                           0.238   Prob(JB):                        0.298
Kurtosis:                       1.877   Cond. No.                         5.41
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Panel regression 

In [38]:
!pip install linearmodels

     |████████████████████████████████| 1.5MB 5.4MB/s 


In [39]:
from linearmodels import PanelOLS, PooledOLS, RandomEffects

In [40]:
date = pd.Categorical(df_clean.date_str)
data = df_clean.set_index(['state', 'date'])
data['date'] = date
data.head()

Name  FIPS  ...      rank        date
state date                             ...                      
MA    2020-01-20  Massachusetts    25  ...  0.223983  01/20/2020
WA    2020-01-20     Washington    53  ...  0.228332  01/20/2020
      2020-01-27     Washington    53  ...  0.247903  01/27/2020
MA    2020-01-27  Massachusetts    25  ...  0.206897  01/27/2020
FL    2020-01-27        Florida    12  ...  0.291395  01/27/2020

[5 rows x 7 columns]

## Check panel regression to see if it finds corr between 2 dependent vars

In [41]:
exog_vars = ['AQI', 'rank']
exog = sm.add_constant(data[exog_vars])

In [42]:
mod = PooledOLS(data.positive, exog)
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:               positive   R-squared:                        1.0000
Estimator:                  PooledOLS   R-squared (Between):              1.0000
No. Observations:                2111   R-squared (Within):               1.0000
Date:                Thu, Dec 10 2020   R-squared (Overall):              1.0000
Time:                        17:44:45   Log-likelihood                 4.022e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                   7.357e+32
Entities:                          51   P-value                           0.0000
Avg Obs:                       41.392   Distribution:                  F(2,2108)
Min Obs:                       31.000                                           
Max Obs:                       47.000   F-statistic (robust):          7.357e+32
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/data.py:98: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning:


Inputs contain missing values. Dropping rows with missing observations.



In [43]:
exog_vars = ['AQI']
exog = sm.add_constant(data[exog_vars])
mod = PooledOLS(data.positive, exog)
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:               positive   R-squared:                        0.0337
Estimator:                  PooledOLS   R-squared (Between):              0.1042
No. Observations:                2111   R-squared (Within):              -0.0188
Date:                Thu, Dec 10 2020   R-squared (Overall):              0.0337
Time:                        17:46:06   Log-likelihood                -3.227e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      73.444
Entities:                          51   P-value                           0.0000
Avg Obs:                       41.392   Distribution:                  F(1,2109)
Min Obs:                       31.000                                           
Max Obs:                       47.000   F-statistic (robust):             73.444
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/data.py:98: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning:


Inputs contain missing values. Dropping rows with missing observations.



Hence, we can describe 10 percent of variance with the Panel regression.
